# Trabajo 3
Alumnos:
- Marcos Hernández Rodríguez
- Salva ***

# Setup

In [119]:
%load_ext autoreload
%autoreload 2

import numpy as np

1. Sobre la base de datos de peatones para clasificación. Aplicar un clasificador SVM. Se deberá implementar HoG (no usar una función ya implementada). Obtener resultados de bondad del clasificador
2. Sobre la base de datos de peatones para clasificación. Aplicar un clasificador usando una red neuronal de convolución. Se deberán reusar y modificar las arquitecturas VGG16, ResNet18 y Xception para tal fin. Obtener una comparativa de bondad usando predictores como agudeza, F1, recall, precision.
3. Sobre la base de datos de peatones para clasificación. Compara la mejor red neuronal obtenida en el punto 2 con el clasificador SVM.

# Read Data

In [17]:
# Preparar los datos para las imágenes con personas andando
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

pedestrians_path = 'input/classification/pedestrians128x64/'
pedestrians_images = load_images_from_folder(pedestrians_path)
print(type(pedestrians_images))
print(len(pedestrians_images))
print(pedestrians_images[0].shape)

<class 'list'>
924


In [23]:
# Select 400 images randomly
import random
pedestrians_images = random.sample(pedestrians_images, 400)
print(type(pedestrians_images))
print(len(pedestrians_images))
print(pedestrians_images[0].shape)

<class 'list'>
400
(128, 64, 3)


In [51]:
# Preparar los datos para las imágenes sin personas andando
no_pedestrians_path = 'input/classification/pedestrians_neg/'
no_pedestrians_images = load_images_from_folder(no_pedestrians_path)
print(type(no_pedestrians_images))
print(len(no_pedestrians_images))
print(no_pedestrians_images[0].shape)

# Select images randomly
import random
no_pedestrians_images = random.sample(no_pedestrians_images, 40)
print(type(no_pedestrians_images))
print(len(no_pedestrians_images))
print(no_pedestrians_images[0].shape)

# Resize de las imagenes a 512x512
import cv2
for i, img in enumerate(no_pedestrians_images):
    no_pedestrians_images[i] = cv2.resize(no_pedestrians_images[i], (512, 512))

    # Escoger un trozo de 128x64
    no_pedestrians_images[i] = no_pedestrians_images[i][192:320, 224:288]

print(no_pedestrians_images[0].shape)
print(len(no_pedestrians_images))

<class 'list'>
50
(256, 256, 3)
<class 'list'>
40
(256, 256, 3)
(128, 64, 3)
40


## Caracterizar la imagen usando Histograma de Orientación de Gradiente (HOG)

In [104]:
ped_img.shape()

TypeError: 'tuple' object is not callable

In [111]:
print(h.get_gradient_matrix(pedestrians_images[0]))

TypeError: 'tuple' object is not callable

In [163]:
from modules import helpers as h

for i, ped_img in enumerate(pedestrians_images):
    print(i)

    ### PRIMERA FASE: Obtener el gradiente de cada píxel
    gradient_matrix = h.get_gradient_matrix(ped_img)
    print(f'gradient_matrix shape: {gradient_matrix.shape}')

    ### SEGUNDA FASE: Obtener la división en células
    print(h.split_in_cells(gradient_matrix, cell_size=8))

    if i == 1:
        break

0
128
<class 'numpy.ndarray'>
(128, 64, 3)
<class 'int'> 128
<class 'int'> 64
gradient_matrix shape: (128, 64)
--- split_in_cells ---
gradient_matrix shape: (128, 64)
cell_rows: 16
cell_columns: 8
(8, 8)
[[  0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.         210.0101188   60.61745504  65.64914972  44.27753381
   36.97018866  14.07351181          nan]
 [  0.         183.129203   145.4547377   82.81660185  75.07481538
    0.                  nan   7.4915149 ]
 [  0.          72.45881961 145.99290123 235.94885364 196.33700234
  188.96447813 243.35265698          nan]
 [  0.                  nan  18.33471059   0.         130.10598891
           nan 185.67603383  35.17703516]
 [  0.         158.47912777 179.06950162   3.99655708   0.
   25.9569329   22.96637544 144.27681633]
 [  0.          66.64457967 173.20914274          nan   2.25258427
  171.71088723  23.96944752 176.54231507]
 [  0.          82.82173674 177.02358147   0.    

c:\Users\marco\Google Drive\02. Máster\12. VisiónComp\Trabajo 3 - Clasificacion y Segmentacion de Peatones (Teoría 4 Puntos)\modules\helpers.py:38: RuntimeWarning: invalid value encountered in sqrt
  norm = np.sqrt(0.5 * (brackets))
c:\Users\marco\Google Drive\02. Máster\12. VisiónComp\Trabajo 3 - Clasificacion y Segmentacion de Peatones (Teoría 4 Puntos)\modules\helpers.py:36: RuntimeWarning: divide by zero encountered in double_scalars
  argument = 0.5 * np.arctan(2*gXY / (gXX-gYY))
c:\Users\marco\Google Drive\02. Máster\12. VisiónComp\Trabajo 3 - Clasificacion y Segmentacion de Peatones (Teoría 4 Puntos)\modules\helpers.py:36: RuntimeWarning: invalid value encountered in double_scalars
  argument = 0.5 * np.arctan(2*gXY / (gXX-gYY))


ValueError: setting an array element with a sequence.

In [145]:
a = np.zeros((4, 4))
print(a)
a[1, 1] = 4
print(a)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 4. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Data split

In [20]:
X=np.concatenate([HOG_X_pos,HOG_X_neg])#Concatenación de HOG
y=np.concatenate([y_pos,y_neg]); #Concatenación de valor de la clase
ntrain= round(0.8*len(y));#cardinal del conjunto entrenamiento
ntest = len(x)-ntrain; #cardinal del conjunto test

idx = np.random.permutation(np.arange(len(y))) #obtenemos una permutación
X_train= X[idx[0:ntrain],:]; #Obtenemos barajado el conjunto entrenamiento
y_train=y[idx[0:ntrain]] #igual para el valor de clase
X_test= X[idx[ntrain+1:len(y)],:];#Obtenemos barajado el conjunto test
y_test=y[idx[ntrain+1:len(y)]]; #igual para el valor de clase


NameError: name 'HOG_X_pos' is not defined

# Classification with SVM

In [ ]:
from sklearn.svm import SVC
m=SVC();
m.fit(X_train,y_train)

y_pred_train=m.predict(X_train); 

y_pred_test=m.predict(X_test); 

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_pred_train)
confusion_matrix(y_test,y_pred_test)

# Classification with CNN

2. Sobre la base de datos de peatones para clasificación. Aplicar un clasificador usando una red neuronal de convolución. Se deberán reusar y modificar las arquitecturas VGG16, ResNet18 y Xception para tal fin. Obtener una comparativa de bondad usando predictores como agudeza, F1, recall, precision.

In [ ]:
# VGG16, ResNet18 y Xception.
# Dijo Sara que se puede usar ResNet50 en vez de la 18 si no funciona bien la 18



3. Sobre la base de datos de peatones para clasificación. Compara la mejor red neuronal obtenida en el punto 2 con el clasificador SVM.

# Semantic Segmentation

Sobre la base de datos de peatones para segmentación semántica. Partiendo de la
arquitectura base probar diferentes modificaciones de esta. Realizar un análisis de las
arquitecturas estudiadas:
- Obtener en este análisis el promedio de IoU (intersection over union) a lo largo del conjunto test sobre las diferentes arquitecturas estudiadas. Obtener la agudeza.
- Tiempo de aprendizaje.
- Dimensión de la representación latente.

In [ ]:
# El alumno/a debe tener en cuenta que las imágenes tienen diferentes dimensiones tanto en el conjunto de entrenamiento como en el conjunto test. Por lo tanto un paso previo será normalizar las imágenes a la misma dimensión.

In [57]:
# Read data training images
data_segmentation_train_imgs_path = 'input/data_segmentation/train/images/'
data_seg_train_imgs = load_images_from_folder(data_segmentation_train_imgs_path)
print(type(data_seg_train_imgs))
print(len(data_seg_train_imgs))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_train_imgs[i].shape)




# Read data training masks
data_segmentation_train_masks_path = 'input/data_segmentation/train/mask/'
data_seg_train_masks = load_images_from_folder(data_segmentation_train_masks_path)
print(type(data_seg_train_masks))
print(len(data_seg_train_masks))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_train_masks[i].shape)

<class 'list'>
96
(406, 612, 3)
(378, 745, 3)
(418, 670, 3)
(436, 786, 3)
(454, 767, 3)
<class 'list'>
96
(406, 612, 3)
(378, 745, 3)
(418, 670, 3)
(436, 786, 3)
(454, 767, 3)


In [59]:
# Read data test images
data_segmentation_test_imgs_path = 'input/data_segmentation/test/images/'
data_seg_test_imgs = load_images_from_folder(data_segmentation_test_imgs_path)
print(type(data_seg_test_imgs))
print(len(data_seg_test_imgs))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_test_imgs[i].shape)




# Read data test masks
data_segmentation_test_masks_path = 'input/data_segmentation/test/mask/'
data_seg_test_masks = load_images_from_folder(data_segmentation_test_masks_path)
print(type(data_seg_test_masks))
print(len(data_seg_test_masks))
for i in [0, 1, 2, 3, 4]:
    print(data_seg_test_masks[i].shape)

<class 'list'>
74
(536, 559, 3)
(414, 455, 3)
(445, 479, 3)
(397, 396, 3)
(344, 335, 3)
<class 'list'>
74
(536, 559, 3)
(414, 455, 3)
(445, 479, 3)
(397, 396, 3)
(344, 335, 3)


In [61]:
print(len(data_seg_train_imgs))
print(len(data_seg_train_masks))

print(len(data_seg_test_imgs))
print(len(data_seg_test_masks))

96
96
74
74


In [99]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import InputLayer, ReLU, Conv3D, Conv3DTranspose, Softmax, Dense

# https://keras.io/api/layers/

# Creamos el modelo
model = tf.keras.Sequential()

# Falla algo de las dimensiones
model.add(InputLayer(input_shape=(214, 214, 3), name='imageinput'))
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding="same", name='conv_1'))
# salida de capa de 214x214x64
model.add(ReLU(name='relu_1'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding=(0,0,0,0), name='maxpool_1'))
# salida de capa de 107x107x64
model.add(tf.keras.layers.Conv3D(64, (3,3,64), strides=(1,1), padding="same", name='conv_2'))
model.add(ReLU(name='relu_2'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding=(0,0,0,0), name='maxpool_2'))
model.add(tf.keras.layers.Conv3D(64, (3,3,64), strides=(1,1), padding="same", name='conv_3'))
model.add(ReLU(name='relu_3'))
model.add(tf.keras.layers.Conv3D(64, (3,3,64), strides=(1,1), padding="same", name='conv_4'))

# Capas decodificadoras
# cropping es lo mismo que padding?
model.add(Conv3DTranspose(64, (4,4,64), strides=(2, 2), padding=(0,0,0,0), name='transposed-conv_1'))
model.add(Conv3DTranspose(64, (4,4,64), strides=(2, 2), padding=(0,0,0,0), name='transposed-conv_2'))

model.add(tf.keras.layers.Conv3D(2, (1,1,64), strides=(1,1), padding=(0,0,0,0), name='conv_5'))
# A continuación aplicamos una capa softmax con función de perdida entropia cruzada para obtener la clases fondo o peatón.
model.add(Softmax(name='softmax'))
# ? es una dense?
model.add(Dense(1, activation='sigmoid', name='classoutput'))



model.summary()


model.compile(optimizer="adam", loss="BinaryCrossentropy")

2.8.0


ValueError: Input 0 of layer "conv_1" is incompatible with the layer: expected min_ndim=5, found ndim=4. Full shape received: (None, 214, 214, 3)